```{autolink-concat}
```

::::{margin}
:::{card} PWA101: Amplitude Analysis with Python basics
TR-033
^^^
This document introduces Amplitude Analysis / Partial Wave Analysis (PWA) by demonstrating its application to a specific reaction channel and amplitude model. Basic Python programming and libraries (e.g. `numpy`, `scipy`, etc.) are used to illustrate the more fundamental steps of PWA in hadron physics.
+++
✅&nbsp;[ComPWA/RUB-EP1-AG#93](https://github.com/ComPWA/RUB-EP1-AG/issues/93), [compwa.github.io#217](https://github.com/ComPWA/compwa.github.io/pull/217)
:::
::::

# Amplitude Analysis 101 (PWA 101)

In [ ]:
%pip install -q "phasespace"==1.9.0 iminuit==2.25.2 matplotlib==3.7.3 numpy==1.24.4 pandas==2.2.1 particle==0.23.0 scipy==1.10.1 vector==1.1.1.post1

:::{admonition} Abstract
This document introduces Amplitude Analysis / Partial Wave Analysis (PWA) by demonstrating its application to a specific reaction channel and amplitude model. It aims to equip readers with a basic understanding of the full workflow and methodologies of PWA in hadron physics through a practical, hands-on example. Only basic Python programming and libraries (e.g. [`numpy`](https://numpy.org/doc/stable), [`scipy`](https://docs.scipy.org/doc/scipy), etc.) are used to illustrate the more fundamental steps in a PWA. Calculations with 4-vectors in this report are performed with the [`vector`](https://vector.readthedocs.io/en/latest/usage/intro.html) package.
:::

:::{seealso}
A follow-up tutorial is prepared in [PWA101 v2.0](https://compwa.github.io/gluex-nstar). Whereas this report focuses on common, numerical Python libraries, v2.0 formulates the amplitude model with a {doc}`symbolic approach</symbolics>`.
:::

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

In [ ]:
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phasespace
import scipy as sp
import tensorflow as tf
import vector
from iminuit import Minuit
from matplotlib import gridspec
from scipy.special import lpmv
from tqdm.auto import tqdm
from vector.backends.numpy import MomentumNumpy4D

## Amplitude model

<!-- cspell:ignore Mathieu -->
This amplitude model is adapted from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/) by Vincent Mathieu.

The photo-production reaction $ \gamma p \to \eta \pi^0 p$ is one of the reaction channels in experiment such as [the GlueX experiment](http://www.gluex.org/). For simplicity, the decays are described by an amplitude model with three possible resonances: $a_2$, $\Delta^+$, and $N^*$. 

```{image} https://github.com/ComPWA/compwa-org/assets/17490173/ec6bf191-bd5f-43b0-a6cb-da470b071630
:width: 100%
```

Given these three subsystems in this particle transition, we can identify three amplitudes $A^{12}$, $A^{23}$, and $A^{31}$:

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \times s_{12}^{0.5+0.9u_3}  \nonumber \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} \times s_{23}^{0.5+0.9t_1}  \nonumber \\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \times s_{31}^{1.08+0.2t_2} 
\end{eqnarray}
$$ (full_model_label)

where $s, t, u$ are the Mandelstam variables $s_{ij}=(p_i+p_j)^2$, $t_i=(p_a-p_i)^2$, and $u_i=(p_b-p_i)^2$, m is the mass, $\Gamma$ is the width, $Y^m_l$ is the spherical harmonics, $\Omega_i$ is the decay angles which is a pair of Euler angles (polar angle $\theta$ and azimuthal angle $\phi$), and $a_i$, $b_i$, and $c_i$ are coefficients

The original full amplitude model from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/) is shown in Equation {eq}`full_model_label`.
*In this report, only the Breit-Wigner and Spherical harmonics terms are kept for doing PWA eventually (The exponential mandelstam variable term is abandoned), as shown in Equation {eq}`BW_SH_label`.*

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}}  \\
\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} 
\end{eqnarray}
$$ (BW_SH_label)

The intensity $I$ that describes our measured distributions is then expressed as a coherent sum of the amplitudes $A^{ij}$:

$$
\begin{eqnarray}
I &=& |A|^2 \nonumber \\
A &=& A^{12} + A^{23} + A^{31}  \\
\end{eqnarray}
$$ (123_label)

where $\quad 1 \equiv \eta , \quad  2 \equiv \pi^0 , \quad 3 \equiv p$.

:::{note}
The ultimate choice of the amplitude model (Equations {eq}`BW_SH_label` and {eq}`123_label`) for PWA in this tutorial consists of three resonances, and each of them are formed by two terms: Breit-Wigner with spherical harmonics ($l = 2, 1, 0$).
:::

:::{important}
The spin of the $\eta$ meson and the $\pi^0$ meson are all 0, but the spin of the proton is not 0, it is spin-$\frac{1}{2}$.
In this amplitude model the **spin** of baryon (proton in this example reaction) is simplified, 
by treating it as a spin-0 particle.
Overall,
the $\eta$, $\pi^0$ and $p$ are all treated as spin-0 particles.

The primary motivation for assuming proton to be spin-0 is to avoid the necessity of aligning the amplitudes. 
This assumption enables the intensity $I$ to be written as a coherent sum of the amplitudes of the subsystems without the need for additional Wigner rotations.

This model assumes that the total intrinsic spin $s$ is ignored, 
meaning that the total angular momentum $J$ of the system or any subsystems within this model solely depends on the orbital angular momentum $L$,
characterized by the quantum number $l$. 
This simplification allows us to use spherical harmonics  $Y_l^m(\theta,\phi)$ more straightforwardly, 
as only the orbital angular momentum component is involved, 
and combinations such as Clebsch-Gordan coefficients are not considered.

In this context, the spherical harmonics are relevant only to the resonances. 
Here, $l$ represents the spin of the resonances, 
and $m$ represents the spin projection. 
The total angular momentum and coupled spin (for the two-body state of the two decay products of the resonance) are not considered. 
According to Richman and other classical references on helicity, 
this is known as the helicity basis, 
as opposed to the canonical basis where both $L$ and $S$ values are used, 
resulting in a more complex coherent sum. 
The transformation between these bases can be found in resources such as [AmpForm](https://ampform.rtfd.io/stable/usage/helicity/formalism.html).

In our case: 
- $A^{12}$ amplitude represents a d-wave interaction, because we assume there is a $a_2$ resonance (spin 2) in this subsystem.
    - The possible $m$ values are $−2,−1,0,1,2$. Each of these values corresponds to different orientations of the d-wave. The wave type is solely determined by $l$ and all these $m$ values still describe d-wave characteristics.
- $A^{23}$ amplitude represents a p-wave interaction, because we assume this subsystem has a (spin-1) $\Delta^+$ resonance.
    - The possible $m$ values are $−1,0,1$. Each of these values corresponds to different orientations of the p-wave. Similarly, these values are all p-wave orientations.
- $A^{31}$ amplitude represents a s-wave interaction, because we assume there is one spin-0 $N^*$ resonance in this subsystem.
    - The only possible $m$ value is 0, which is consistent with the spherical symmetry of s-waves.
:::

## Number of events

The number of events for phase space generation and data generation can be changed in this section.

In [ ]:
phsp_events = 500_000
data_events = 250_000

## Phase space generation

In this section, the phase space of this reaction is generated using [`phasespace`](https://github.com/zfit/phasespace) python package.

Phase space represents the set of all possible states (positions and momenta) of a system.
Understanding phase space is essential for analyzing particle collisions, decays, and interactions. It involves calculating the likelihood of different final states from given initial conditions while obeying conservation laws (energy, momentum, angular momentum, etc.). The available phase space volume for a process directly correlates with the probability of that process occurring.

In the following, variables that are not labeled with "lab" are in the CM frame.

Firstly, some kinematic considerations. In Center-of-Mass (CM) frame the 4-momentum of the total system can be acquired by 4-momentum conservation:

$$\begin{pmatrix}
 E_0 \\
 0 \\
 0 \\
 0
\end{pmatrix}
= \begin{pmatrix}
 E_{\gamma} \\
 0 \\
 0 \\
 p_z
\end{pmatrix} +
\begin{pmatrix}
 E_p \\
 0 \\
 0 \\
 -p_z
\end{pmatrix}
= \begin{pmatrix}
 E_{\eta} \\
 p_{\eta,x} \\
 p_{\eta,y} \\
 p_{\eta,z}
\end{pmatrix} +
\begin{pmatrix}
 E_{\pi} \\
 p_{\pi,x} \\
 p_{\pi,y} \\
 p_{\pi,z}
\end{pmatrix} +
\begin{pmatrix}
 E_p \\
 p_{p,x} \\
 p_{p,y} \\
 p_{p,z}
\end{pmatrix}
$$ (4momentum_label)

For the record, we need to add parameters of beam momentum (or beam energy) in the lab frame  and CM total energy

:::{caution}
The calculation here involved using the values from CM frame. While this frame is commonly used for theoretical calculations, experimental data is often analyzed in the lab frame. However, it's worth noting that for collider experiments, the CM frame can coincide with the lab frame.
:::

The [GlueX](http://www.gluex.org/) experiment at Jefferson Lab uses a fixed proton target with a linearly polarized photon beam, and the beam energy range in the lab frame is typically from [**8 to 9 GeV**](https://doi.org/10.7566/JPSCP.26.022002).

We use the $\gamma$ beam energy in the lab frame as input:

$$
E_{\gamma, \text{lab}} = 8.5 \; \text{GeV}
$$ (beam_energy_label)

From this, we can calculate the energy of the gamma in the CM frame as follows.

The four-momentum of photon (beam) in the lab frame:

$$
p_{\gamma,\text{lab}} = (E_{\gamma, \text{lab}}, \vec{p}_{\gamma,\text{lab}})
$$ (beam_four_momentum)

Since photon is massless, 

$$
m_{\gamma}=0
$$ (massless_photon)

with 

$$
m^2 = E^2 - \vec{p}^2
$$ (four_momentum_conservation)

and thus,

$$
E_{\gamma} = |\vec{p_{\gamma}}| .
$$ (gamma_energy)

The four-momentum of proton (target) in the lab frame:

$$
p_{p,\text{lab}} = (m_p, \vec{0})
$$ (four_momentum_target_lab)

where $m_p$ is the proton mass.

We have the total four-momentum in lab frame:

$$
p_{\text{tot},\text{lab}} = p_{\gamma,\text{lab}} + p_{p,\text{lab}}
$$ (total_four_momentum_lab_frame)

$$
E_{p} = \sqrt { p_{p}^2+ m_{p}^2}
$$ (label_p_four_momentum)

From the **lab frame** perspective, the CM total energy with expression in quantities from the **lab frame** is thus:

$$
\sqrt{s} = E_0 = m_0 = \sqrt{2 E_{\gamma,\text{lab}} m_p + m_p^2}
$$ (lab_frame_CM_total_energy)

Equivalently, from the **CM frame** perspective, since $\vec{p}_{\gamma} = -\vec{p}_{p}$ and $|\vec{p}_{\gamma}| = |\vec{p}_{p}|= p_{z}$, we can find out the CM total energy with expression in quantities from the CM frame is:

$$
\sqrt{s} = E_0 = m_0 = |p_{z}|+\sqrt{p_{z}^2 + m_p^2}
$$ (total_energy_CM_frame_still)

Therefore $\gamma$ energy ($E_{\gamma}$) and $\gamma$ beam momentum ($p_{\gamma}$) in **CM frame** can be found,they are:

$E_{\gamma} = |\vec{p}_{\gamma}| = p_{z} = 1.943718 \; GeV$

Our implementation based on Equation {eq}`lab_frame_CM_total_energy` thus becomes:

In [ ]:
E_lab_gamma = 8.5
m_proton = 0.938
m_0 = np.sqrt(2 * E_lab_gamma * m_proton + m_proton**2)
m_eta = 0.548
m_pi = 0.135

In [ ]:
p_beam = 1.943718
m_0test = p_beam + np.sqrt(p_beam**2 + m_proton**2)

In [ ]:
m_0

In [ ]:
m_0test

Thus, we then have the mass of the system $m_0$ (or $m_{p\gamma}$ in this case) in CM frame:

$$
E_{0} = m_0\approx 4.102 \;\; \text{GeV} 
$$

:::{note}
The [`phasespace`](https://github.com/zfit/phasespace) library is a Python package designed to simulate particle decays according to the principles of relativistic kinematics and phase space distributions. 

We use the [`phasespace`](https://github.com/zfit/phasespace) to generate decay particles (4-momentum and weights):
:::

In [ ]:
weights, particles = phasespace.nbody_decay(m_0, [m_eta, m_pi, m_proton]).generate(
    n_events=phsp_events
)

:::{admonition} **weights**:
the statistical weights for each generated event. These weights represent how likely each particular event configuration (set of momenta and energies) is, based on phase space considerations.
:::

:::{admonition} **`particles`**:
this (python array) contains the information of the generated four-momenta (energy and momentum components) of the decay products for each event. 
:::

### 4 momentum of decay particles

The following code defines some functions for generating a phase space sample of 4-momenta.

In [ ]:
pγ_mass = m_0
eta_mass = m_eta
pi_mass = m_pi
p_mass = m_proton
n_final_state = 3

:::{tip}
The function block below is scrollable.
:::

In [ ]:
def generate_phsp_decay(
    size: int, seed: int | None = None, bunch_size: int | None = None
) -> tuple[MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D]:
    if bunch_size is None:
        bunch_size = size
    rng = np.random.default_rng(seed=seed)
    generated_seed = rng.integers(1_000_000)
    phsp_sample = generate_phsp_bunch(size, generated_seed)
    while get_size(phsp_sample) < size:
        bunch = generate_phsp_bunch(bunch_size, generated_seed)
        phsp_sample = concatenate(phsp_sample, bunch)
    phsp_sample = remove_overflow(phsp_sample, size)
    return tuple(to_vector(tensor) for tensor in phsp_sample)


def generate_phsp_bunch(
    size: int, seed: int | None = None
) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    rng = np.random.default_rng(seed=seed)
    weights, particles = phasespace.nbody_decay(
        pγ_mass, [eta_mass, pi_mass, p_mass]
    ).generate(n_events=size, seed=seed)
    random_weights = rng.uniform(0, weights.numpy().max(), size=weights.shape)
    selector = weights > random_weights
    return tuple(particles[f"p_{i}"][selector] for i in range(n_final_state))


def get_size(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor]):
    return len(phsp[0])


def concatenate(
    phsp1: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
    phsp2: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    return tuple(tf.concat([phsp1[i], phsp2[i]], axis=0) for i in range(3))


def remove_overflow(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor], size: int):
    return tuple(tensor[:size] for tensor in phsp)


def to_vector(tensor: tf.Tensor) -> MomentumNumpy4D:
    return vector.array({
        key: tensor.numpy().T[j] for j, key in enumerate(["px", "py", "pz", "E"])
    })

A few functions are defined above for generating phase space samples of particle decays and converting them into four-momentum vectors. The main ideas of them are:

:::{note}
- Step 1, Generate Phase Space Sample (`generate_phsp_bunch()`)
    - Random Number Generator

      A NumPy random number generator (`rng`) is initialized with a fixed seed for reproducibility. This ensures that every time the simulation is run, it produces the same random numbers, which is crucial for debugging and validating simulation results.
      
    - Phase Space Generation

 
      The `phasespace` package is used to simulate a decay process where a parent particle decays into three daughter particles. This is set up by specifying the masses of the parent particle (`pγ_mass`) and the daughter particles (`eta_mass`, `pi_mass`, `p_mass`). The function `nbody_decay` followed by generate is used to create the phase space and return both the weights of each decay event and the momenta of the decay products.
 
      
    - Rejection Sampling
 
      To refine the sample, rejection sampling is applied. The `phasespace` package generates four-momenta that are not evenly distributed. The distributions appear even only when multiplied by the generated weights. To achieve this, a random set of weights (`random_weights`) is generated, and only those events where the original weight exceeds the random weight are selected. 
 
      
      
- Step 2, Ensure Sufficient Sample Size (`generate_phsp_decay()`)
    - Iterative Sampling
        
      The function starts by generating an initial phase space sample. If this initial sample does not meet the required size, more samples are generated and concatenated until the cumulative size is adequate.
      
    - Removing Excess

 
      Once the target sample size is reached or exceeded, `remove_overflow()` trims the sample to precisely match the requested size.

- Step 3, Convert to [Four-Momentum Vectors](https://vector.readthedocs.io/en/latest/api/backends/vector.backends.numpy.html#vector.backends.numpy.MomentumNumpy4D) (`to_vector()`)
    - Four-Momentum Representation

      For each tensor in the phase space tuple (representing the momentum components like px, py, pz), the `to_vector` function converts them into a structured array (`MomentumNumpy4D`). This structured array includes keys for each momentum component and possibly the energy component, arranged as needed for subsequent physics analysis.
:::

In [ ]:
%%time
p1_phsp_test, p2_phsp_test, p3_phsp_test = generate_phsp_decay(
    size=phsp_events, seed=42
)

The time is for reference of this phase space generation algorithm (see also the section of [data sample generation](#data-generation)).

### 4 momentum of initial state particles

The initial state, $\gamma$ (a) and $p$ (b), are generated.

To find $p_a$ and $p_b$ by 4-momentum conservation, energy conservation in this case.

Given that:

```{math}
:label: m_a_m_b_p_a_x_p_a_z

m_a = m_{\gamma} =0 \\

m_b = m_p \\

p_{a,x} = p_{a,y} = 0 = p_{b,x} = p_{b,y} \\

p_{a,z} = - p_{b,z}

```

Due to energy conservation, we have:

```{math}
:label: E_a_+_E_b_E_0

E_a + E_b = E_1 + E_2 + E_3 = E_0 \\

E_a+ E_b = \sqrt{m_a^2 + p_a^2} + \sqrt{m_b^2 + p_b^2} = E_0

```

Since $m_a=0$ and $p_a = -p_b$, thus
```{math}
:label: p_a_+_m_b_2_-_p_a_2

p_a + \sqrt{m_b^2 + (-p_a)^2} = E_0
```

Reorganizing, we get,
```{math}
:label: p_a_z_m_p_2_p_a_2

p_{a,z} + \sqrt{m_p^2 + p_a^2} - E_0 = 0
```
thus

```{math}
:label: p_a_z_E_0_2_-_m_p_2

p_{a,z} = \frac{E_0^2 - m_p^2}{2E_0}
```

then

```{math}
:label: label_p_b_E_a_E_b

p_{b,z} = -p_{a,z} \\

E_a =p_a \\

E_{b} = \sqrt{m_b^2+p_b^2}
```

And now we can implement the function `compute_pa_pb()` to compute four-momentum of $p_a$ and $p_b$ based on Equations {eq}`label_p_b_E_a_E_b`.

In [ ]:
def compute_pa_pb(
    p1_phsp: MomentumNumpy4D, p2_phsp: MomentumNumpy4D, p3_phsp: MomentumNumpy4D
) -> tuple[MomentumNumpy4D, MomentumNumpy4D]:
    shape = p1_phsp.shape
    E_0 = (p1_phsp + p2_phsp + p3_phsp).e
    px = np.zeros(shape)
    py = np.zeros(shape)
    pz = np.ones(shape) * (E_0**2 - p3_phsp.m.mean() ** 2) / (2 * E_0)
    E = np.ones(shape) * np.sqrt((p3_phsp.m.mean()) ** 2 + pz.mean() ** 2)
    pa_phsp = MomentumNumpy4D({"E": pz, "px": px, "py": py, "pz": pz})
    pb_phsp = MomentumNumpy4D({"E": E, "px": px, "py": py, "pz": -pz})
    return pa_phsp, pb_phsp

In [ ]:
%%time
pa_phsp_test, pb_phsp_test = compute_pa_pb(p1_phsp_test, p2_phsp_test, p3_phsp_test)

### 4 momentum of all particles

After testing the feasibility of the functions (`generate_phsp_decay()` and `compute_pa_pb()`) to generate the four-momentum of decay particles in phase space and computing the corresponding four-momentum of reaction particles.
We now create a function `generate_phsp_all()` to create and compute all particles in phase space all-in-once (combine the two functions previously all into this function).

In [ ]:
def generate_phsp_all(
    size: int, seed: int | None = None, bunch_size: int | None = None
) -> tuple[
    MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D
]:
    p1_phsp, p2_phsp, p3_phsp = generate_phsp_decay(size, seed, bunch_size)
    pa_phsp, pb_phsp = compute_pa_pb(p1_phsp, p2_phsp, p3_phsp)
    return p1_phsp, p2_phsp, p3_phsp, pa_phsp, pb_phsp

In [ ]:
%%time
p1_phsp, p2_phsp, p3_phsp, pa_phsp, pb_phsp = generate_phsp_all(
    size=phsp_events, seed=42
)

In [ ]:
p0_phsp = pa_phsp + pb_phsp
p12_phsp = p1_phsp + p2_phsp
p23_phsp = p2_phsp + p3_phsp
p31_phsp = p3_phsp + p1_phsp

s12_phsp = p12_phsp.m2
s23_phsp = p23_phsp.m2
s31_phsp = p31_phsp.m2
t1_phsp = (pa_phsp - p1_phsp).m2
t2_phsp = (pa_phsp - p2_phsp).m2
u3_phsp = (pb_phsp - p3_phsp).m2

In [ ]:
np.testing.assert_almost_equal(
    s12_phsp,
    (p1_phsp.e + p2_phsp.e) ** 2
    - (
        p1_phsp.p**2
        + p2_phsp.p**2
        + 2
        * (p1_phsp.px * p2_phsp.px + p1_phsp.py * p2_phsp.py + p1_phsp.pz * p2_phsp.pz)
    ),
)

np.testing.assert_almost_equal(p0_phsp.p2.max(), 0)

np.testing.assert_almost_equal(
    t1_phsp,
    (pa_phsp.e - p1_phsp.e) ** 2
    - (
        pa_phsp.p**2
        + p1_phsp.p**2
        - 2
        * (pa_phsp.px * p1_phsp.px + pa_phsp.py * p1_phsp.py + pa_phsp.pz * p1_phsp.pz)
    ),
)
np.testing.assert_almost_equal(
    t2_phsp,
    (pa_phsp.e - p2_phsp.e) ** 2
    - (
        pa_phsp.p**2
        + p2_phsp.p**2
        - 2
        * (pa_phsp.px * p2_phsp.px + pa_phsp.py * p2_phsp.py + pa_phsp.pz * p2_phsp.pz)
    ),
)
np.testing.assert_almost_equal(
    u3_phsp,
    (pb_phsp.e - p3_phsp.e) ** 2
    - (
        pb_phsp.p**2
        + p3_phsp.p**2
        - 2
        * (pb_phsp.px * p3_phsp.px + pb_phsp.py * p3_phsp.py + pb_phsp.pz * p3_phsp.pz)
    ),
)

### Dalitz Plot of Phase Space

The sample is plotted to check whether the distribution looks even in the Dalitz plane (as shown below it is!).

Following this in later sections, we will present Dalitz plots for both experimental data and the fitted model. These plots will allow us to compare the theoretical predictions with actual observations, and to assess the accuracy and effectiveness of the fitted model in describing the decay process.

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("Phase Space Dalitz Plot")
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05])  # The last column for colorbar
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
cax = plt.subplot(gs[2])  # For colorbar

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    cmin=1e-2,
    density=True,
    cmap="jet",
    range=[[0, 10.3], [0.5, 13]],
)
ax2.set_title("2D Histogram")
ax2.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")

scat1 = ax1.scatter(s12_phsp, s23_phsp, s=1e-4, c="black", norm=mpl.colors.Normalize())
ax1.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax1.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
ax1.set_title("Scatter Plot")

cbar2 = fig.colorbar(hist2[3], cax=cax)
fig.tight_layout()
fig.show()

<!-- cspell:ignore colour -->
:::{tip}
There are different ways to represent the Dalitz plot, each with its advantages.

- *Scatter Plot*: This method plots individual events as points, offering a clear view of the density and distribution of events within the phase space. It is particularly useful for visualizing smaller datasets or when high resolution is needed to identify specific features or clusters.

- *2D Histogram*: This approach divides the phase space into bins and counts the number of events within each bin, representing the density of events using a colour scale. It is effective for large datasets, providing a smooth and continuous representation of the phase space that highlights overall trends and structures.

In conclusion, to check if the phase space sample is evenly distributed, a scatter plot is typically more straightforward and visually clear.
:::

## Calculation of Angles

Before introducing CM and helicity angles, we first introduce **polar angles** and **azimuthal angles** in spherical coordinates.

In the spherical coordinates, the **polar angles**:

```{math}
:label: polar_angle_sh
\theta = \arccos \frac{p_z}{|p|}
```

In spherical coordinates  are **azimuthal angles**:

```{math}
:label: azimuthal_angle_sh
\phi = \arctan2(p_y , p_x)
```

```{image} https://github.com/ComPWA/gluex-nstar/assets/17490173/eca119ff-0dfc-4fbf-8868-48cde3db01d0
:width: 40%
```

In Equation {eq}`polar_angle_sh`, $p_z$ is equivalent to $z$, and $|p|$ is $r$ in figure above.

In Equation {eq}`azimuthal_angle_sh`, $p_y$ equivalent to $y$, and $p_x$ is $x$  in figure above.

### CM Angles 

Angles in the CM frame (CM Angles) are the polar and azimuthal angles in the phase space generation and also in the data generation in the CM frame (i.e. the frame that satisfies the relations in Equation {eq}`4momentum_label`), they are different than the helicity angles in the subsystem (which is after rotation and boost into the subsystem).

The values for phase space can be obtained directly in the CM frame, without boosting into a different frame after the generation.
We denote these angles as:

$(\theta_1^{\text{CM}}, \theta_2^{\text{CM}}, \theta_3^{\text{CM}}, \phi_1^{\text{CM}}, \phi_2^{\text{CM}}, \phi_3^{\text{CM}})$:

In [ ]:
theta1_CM_phsp = p1_phsp.theta
theta2_CM_phsp = p2_phsp.theta
theta3_CM_phsp = p3_phsp.theta
phi1_CM_phsp = p1_phsp.phi
phi2_CM_phsp = p2_phsp.phi
phi3_CM_phsp = p3_phsp.phi

### Helicity angles

:::{admonition} **Helicity formalism**

The helicity formalism is a crucial framework or powerful tool in nuclear and particle physics for describing and analyzing the spin states of particles in reactions and decays. 

It simplifies the analysis of complex processes by projecting the particle's spin onto its direction of motion. This formalism allows for a clear understanding of angular distributions and polarization effects, facilitating the calculation of **transition amplitudes**, which can provide useful insight into particle interaction and dynamics.

Through helicity states and the use of **Wigner D-matrices**, physicists can effectively interpret experimental data and theoretical models in a more streamlined and intuitive manner.
:::

<!-- cspell:ignore eigenstates -->
:::{note}
**Helicity definition**:

Helicity ($\lambda$) is the projection of a particle's spin ($S$) onto its direction of motion ($p$). Mathematically, it's given by:
```{math}
\lambda = \frac{S \cdot p}{|p|}
```
For massless particles, helicity is a Lorentz-invariant quantity, meaning it remains unchanged under boosts along the direction of motion.

**Helicity states**:

Helicity states are eigenstates of the helicity operator. For a particle moving in the z-direction, the helicity operator is 
```{math}
S \cdot \hat{p}
```
These states simplify the description of particle interactions.

**Wigner D-matrices**:

Wigner D-matrices are used in the helicity formalism to describe the rotation of spin states. 
These matrices depend on the Euler angles of the rotation and are denoted by 
```{math}
D^j_{m'm} (α,β,γ) = e^{-im'α} d^j_{m'm}(β) e^{-imγ},
```
 where $j$ is the spin, $m'$ and $m$ are the magnetic quantum numbers, and
 ```{math}
d^j_{m'm}(β) = D^j_{m'm} (0,β,0)
```
is an element of the orthogonal Wigner's (small) d-matrix.

They play a crucial role in transforming between different reference frames and in calculating the angular dependence of transition amplitudes.

**Transition Amplitudes**:

The transition amplitude for a process is expressed in terms of helicity states. 
This reduces the complexity of the calculations as it aligns the spins of the particles with their momenta.
For example, the decay of a particle into two others can be described by the helicity amplitudes, which are functions of the helicities of the initial and final state particles.
:::

:::{seealso}
[Section of helicity formalism in TR-015(Spin alignment implementation)](https://compwa.github.io/report/015.html#helicity-formalism)

[Helicity versus canonical in Ampform](https://ampform.readthedocs.io/stable/usage/helicity/formalism.html)
:::

:::{admonition} Production and decay plane
The **production plane** is defined by the momenta of the incoming particles that participate in the production of a particular state or particle. For example, in a collision between two particles 
A and 
B producing a particle 
C, the production plane is spanned by the momenta of 
A and 
B.
The orientation of this plane is crucial for understanding the initial state interactions and the overall reaction dynamics.

The **decay plane** is defined by the momenta of the decay products of a particle.
For instance, if a particle 
C decays into particles 
D and 
E, the decay plane is spanned by the momenta of 
D and 
E.
Analyzing the decay plane helps in understanding the decay mechanism, angular distributions, and possible correlations between the decay products.
:::

:::{important}

The helicity angles $\Omega_i$ are pairs of Euler angles $\Omega_i = \left(\theta_i, \phi_i\right)$:
- Opening angle $\theta_1 \equiv \theta^{12}_1$ of four-momentum $p_1$ is the angle between $p'_1 \equiv p^{12}_1$ and $p_{12}$ or $-p_3$
    - which means these are the angles between decay products in general.
- The angle $\phi_1 \equiv \phi^{12}_1$ defines the angle between the **production plane** spanned by $p_{12}$ and $p_3$ and the **decay plane** spanned by $p^{(\prime)}_1$ and $p^{(\prime)}_2$.
    - which means these are the angles between the production and decay plane.

- The helicity angles of the other subsystems are defined by cyclic permutation of the indices, such that $\theta_2 \equiv \theta^{12}_2$ and $\theta_3 \equiv \theta^{31}_1$.


These angles are in helicity formalism, the calculation involved rotation and boost into the subsystem as follows.
:::

Before boosting (in CM frame):

<iframe src="https://www.geogebra.org/3d/dgjn83pb?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

After Boosting into system$_{12}$ rest frame:

<iframe src="https://www.geogebra.org/3d/tv5kr8pp?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

:::{note}
It is similar image for boosting into system$_{23}$ and system$_{31}$.
:::

To calculate the helicity angles $\theta$ and $\phi$, a combination of boost and rotation (in Y and Z axis) in the CM frame is defined:

In [ ]:
def theta_helicity(p_i: MomentumNumpy4D, p_ij: MomentumNumpy4D):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(-p_ij.beta).theta


def phi_helicity(p_i: MomentumNumpy4D, p_ij: MomentumNumpy4D):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(-p_ij.beta).phi

In [ ]:
theta1_phsp = theta_helicity(p1_phsp, p12_phsp)
theta2_phsp = theta_helicity(p2_phsp, p23_phsp)
theta3_phsp = theta_helicity(p3_phsp, p31_phsp)
phi1_phsp = phi_helicity(p1_phsp, p12_phsp)
phi2_phsp = phi_helicity(p2_phsp, p23_phsp)
phi3_phsp = phi_helicity(p3_phsp, p31_phsp)

The distribution of angular projection and invariant mass in phase space that discussed in this [section](#my_section) will be shown later, together with data and weighted phase space (model) after data generation.

In [ ]:
def plot_helicity_angles_2d(
    phi1, phi2, phi3, theta1, theta2, theta3, title: str
) -> None:
    fig, axes = plt.subplots(figsize=(13, 4), ncols=3, sharey=True)
    for i, ax in enumerate(axes, 1):
        ax.set_xlabel(Rf"$\theta_{i}$")
        ax.set_ylabel(Rf"$\phi_{i}$")
    axes[0].hist2d(theta1[~np.isnan(phi1)], phi1[~np.isnan(phi1)], bins=100)
    axes[1].hist2d(theta2[~np.isnan(phi2)], phi2[~np.isnan(phi2)], bins=100)
    axes[2].hist2d(theta3[~np.isnan(phi3)], phi3[~np.isnan(phi3)], bins=100)
    fig.suptitle(title)
    fig.tight_layout()
    plt.show()


plot_helicity_angles_2d(
    phi1_phsp,
    phi2_phsp,
    phi3_phsp,
    theta1_phsp,
    theta2_phsp,
    theta3_phsp,
    title="Phase space",
)

The CM angles, helicity angles (in subsystem), and invariant mass in phase space are investigated:

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)

for i, ax in enumerate(thetaCM_ax, 1):
    ax.set_title(Rf"$\cos (\theta_{i}^{{CM}})$")
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phiCM_ax, 1):
    ax.set_title(Rf"$\phi_{i}^{{CM}}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\cos (\theta_{i}^{{{i}{(i % 3 + 1)}}})$")
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}^{{{i}{(i % 3 + 1)}}}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")


thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    label="phsp",
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    label="phsp",
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    label="phsp",
)

phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=50,
    label="phsp",
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=50,
    label="phsp",
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=50,
    label="phsp",
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    label="phsp",
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    label="phsp",
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    label="phsp",
)

phi_ax[0].hist(
    phi1_phsp,
    bins=50,
    label="phsp",
)
phi_ax[1].hist(
    phi2_phsp,
    bins=50,
    label="phsp",
)
phi_ax[2].hist(
    phi3_phsp,
    bins=50,
    label="phsp",
)

mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    label="phsp",
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    label="phsp",
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    label="phsp",
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.tight_layout()
plt.show()

## Spherical harmonics for the model

The calculation of $Y_l^m(\phi, \theta)$ is done via [`scipy.special.sph_harm(m, l, phi, theta)`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html) 

We have changed the use of symbol $\phi$ and $\theta$ according to the more common used definition in physics.

Here the notation of $\theta$ and $\phi$ are not using the same as in `scipy` special function [scipy.special.sph_harm()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html)

(in `scipy`, $\theta$ is the azimuthal  from -$\pi$ to $\pi$ and $\phi$ is the polar angle from 0 to $\pi$)

where the notation we used here is more common in physics:
- $\phi$ is the azimuthal  from -$\pi$ to $\pi$ in this document (where in `scipy` it is represented as $\theta$ and ranged from 0 to $2\pi$)

- $\theta$ is the polar angle from 0 to $\pi$ in this document (where in `scipy` it is represented as $\phi$ with the same range)

```{math}
:label: spherical_harmonics_def
Y_l^m(\phi, \theta) = \sqrt{\frac{2n+1}{4\pi}\frac{(n-m)!}{(n+m)!}}e^{im\phi}P_l^m(\cos(\theta))
```

In the following, we accordingly define functions to compute three terms of spherical harmonics in Equation {eq}`BW_SH_label`, and the explicit form of spherical harmonics as shown in Equation {eq}`spherical_harmonics_def`.

Firstly, the term in $A^{12}$:

In [ ]:
def compute_spherical_harmonics12(
    theta: np.ndarray,
    phi: np.ndarray,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    **kwargs,
) -> np.ndarray:
    return (
        a_plus2 * sp.special.sph_harm(2, 2, phi, theta)
        + a_plus1 * sp.special.sph_harm(1, 2, phi, theta)
        + a_0 * sp.special.sph_harm(0, 2, phi, theta)
        + a_minus1 * sp.special.sph_harm(-1, 2, phi, theta)
        + a_minus2 * sp.special.sph_harm(-2, 2, phi, theta)
    )

The default values of coefficients $a_i$, $b_i$, and $c_i$ in the spherical harmonics:

In [ ]:
initial_SH_parameters = dict(
    a_minus2=0,
    a_minus1=0.5,
    a_0=3.5,
    a_plus1=4,
    a_plus2=2.5,
    b_minus1=-1.5,
    b_0=4,
    b_plus1=0.5,
    c_0=2.5,
)

Assign the initial default values for $a_m$

In [ ]:
a_minus2 = initial_SH_parameters.get("a_minus2")
a_minus1 = initial_SH_parameters.get("a_minus1")
a_0 = initial_SH_parameters.get("a_0")
a_plus1 = initial_SH_parameters.get("a_plus1")
a_plus2 = initial_SH_parameters.get("a_plus2")

In [ ]:
PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=1_000),
    np.linspace(0, np.pi, num=1_000),
)
Z12 = compute_spherical_harmonics12(
    PHI,
    THETA,
    **initial_SH_parameters,
)

We now have a look at the real part and imaginary part of $\sum a_m Y_2^m (\Omega_1)$ in Equation {eq}`BW_SH_label`.

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z12.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z12.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Re($\sum a_m Y_2^m (\Omega_1)$)")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Im($\sum a_m Y_2^m (\Omega_1)$)")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
plt.hist2d(
    p12_phsp.phi,
    p12_phsp.theta,
    bins=100,
    weights=compute_spherical_harmonics12(
        p12_phsp.phi, p12_phsp.theta, a_minus2, a_minus1, a_0, a_plus1, a_plus2
    ).real,
    cmap=plt.cm.coolwarm,
)
plt.title("$p_{12}$ with real part of spherical harmonics as weights in histogram")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\theta$")
plt.show()

Next, the term in $A^{23}$

In [ ]:
def compute_spherical_harmonics23(
    theta: np.ndarray, phi: np.ndarray, b_minus1, b_0, b_plus1, **kwargs
) -> np.ndarray:
    return (
        b_plus1 * sp.special.sph_harm(1, 1, phi, theta)
        + b_0 * sp.special.sph_harm(0, 1, phi, theta)
        + b_minus1 * sp.special.sph_harm(-1, 1, phi, theta)
    )

Assign the initial default values for $b_m$

In [ ]:
b_minus1 = initial_SH_parameters.get("b_minus1")
b_0 = initial_SH_parameters.get("b_0")
b_plus1 = initial_SH_parameters.get("b_plus1")

In [ ]:
Z23 = compute_spherical_harmonics23(PHI, THETA, **initial_SH_parameters)

We then have a look at the real part and imaginary part of $\sum b_m Y_1^m (\Omega_2)$ in equation {eq}`BW_SH_label`.

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z23.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z23.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Re($\sum b_m Y_1^m (\Omega_2)$)")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Im($\sum b_m Y_1^m (\Omega_2)$)")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

Lastly, the corresponding term in $A^{31}$

In [ ]:
def compute_spherical_harmonics31(
    theta: np.ndarray, phi: np.ndarray, c_0, **kwargs
) -> np.ndarray:
    return c_0 * sp.special.sph_harm(0, 0, phi, theta)

Assign the initial default value for $c_0$

In [ ]:
c_0 = initial_SH_parameters.get("c_0")

:::{admonition} **Extra Information**

Alternatively,
we can also calculate the spherical harmonics using the Wigner d-function,
and the Wigner d-function can be related to the associated Legendre polynomials

$Y_l^m(\theta,\phi) = \sqrt{\frac{2l+1}{4\pi}\frac{(l-m)!}{(l+m)!}}e^{im\phi} d^l_{m0}(\theta)$
:::

In [ ]:
def wigner_d_function(j, m, n, beta):
    """
    Calculate the Wigner d-function for given j, m, n, and beta.

    Parameters:
    j (int): Total angular momentum quantum number.
    m, n (int): Magnetic quanta um numbers.
    beta (float): The angle (in radians).

    Returns:
    float: The value of the Wigner d-function.
    """
    # The Wigner d-function can be related to the associated Legendre polynomials
    # Here, we use scipy's lpmv function which computes the associated Legendre polynomials
    return (
        np.sqrt(
            (np.math.factorial(j + m) * np.math.factorial(j - m))
            / (np.math.factorial(j + n) * np.math.factorial(j - n))
        )
        * np.cos(beta / 2) ** (2 * j + n - m)
        * np.sin(beta / 2) ** (m - n)
        * lpmv(abs(n - m), 2 * j, np.cos(beta))
    )


def spherical_harmonics(l_num, m, theta, phi):
    """
    Calculate the spherical harmonics using the Wigner d-function.

    Parameters:
    l (int): Angular momentum quantum number.
    m (int): Magnetic quantum number.
    theta (float): Polar angle in radians.
    phi (float): Azimuthal angle in radians.

    Returns:
    complex: The value of the spherical harmonic.
    """
    # Calculating the spherical harmonics using Wigner d-function
    # Y^m_l(θ, φ) = √((2l+1)/(4π)) * e^(-imφ) * d^l_{m0}(θ)
    normalization = np.sqrt((2 * l_num + 1) / (4 * np.pi))
    return normalization * np.exp(1j * m * phi) * wigner_d_function(l_num, m, 0, theta)

In [ ]:
sp.special.sph_harm(1, 1, phi1_phsp, theta1_phsp)

In [ ]:
spherical_harmonics(1, 1, theta1_phsp, phi1_phsp)

## Implementation of the models

### Toy model parameter values

The toy model parameter values can be obtained from the data file from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/).
But the values are modified here to make the strides in the Dalitz plot more visible.

The purpose is to define and test the model from fewer to more terms in Equation {eq}`BW_SH_label`.

The initial toy model parameter are resonance mass $(M)$ and width $(\Gamma)$:

```{math}
:label: m_gamma_para
M_{12} = M_{\eta\pi^0} = M_{a_2} \\

M_{23} = M_{\pi^0p} = M_{\Delta^+} \\

M_{31} = M_{\eta p} = M_{N^*} \\

\Gamma_{12} = \Gamma_{\eta\pi^0} = \Gamma_{a_2} \\

\Gamma_{23} = \Gamma_{\pi^0p} = \Gamma_{\Delta^+} \\

\Gamma_{31} = \Gamma_{\eta p} = \Gamma_{N^*}
```

In [ ]:
toy_parameters = dict(
    M12=1.32,
    Gamma12=0.1,
    M23=1.24 + 0.3,
    Gamma23=0.1,
    M31=1.57 + 0.3,
    Gamma31=0.1,
    a_minus2=initial_SH_parameters.get("a_minus2"),
    a_minus1=initial_SH_parameters.get("a_minus1"),
    a_0=initial_SH_parameters.get("a_0"),
    a_plus1=initial_SH_parameters.get("a_plus1"),
    a_plus2=initial_SH_parameters.get("a_plus2"),
    b_minus1=initial_SH_parameters.get("b_minus1"),
    b_0=initial_SH_parameters.get("b_0"),
    b_plus1=initial_SH_parameters.get("b_plus1"),
    c_0=initial_SH_parameters.get("c_0"),
)

M12 = toy_parameters.get("M12")
Gamma12 = toy_parameters.get("Gamma12")
M23 = toy_parameters.get("M23")
Gamma23 = toy_parameters.get("Gamma23")
M31 = toy_parameters.get("M31")
Gamma31 = toy_parameters.get("Gamma31")

### Breit-Wigner (only) Model

```{admonition} Reminder
```{math}
:label: B_W_model_only
I = 
\left\lvert \frac{1}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} +
\frac{1}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} +
\frac{1}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \right\rvert ^2
```
```

In [ ]:
def BW_model(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31, **kwargs):
    A12 = BW(s12, M12, Gamma12)
    A23 = BW(s23, M23, Gamma23)
    A31 = BW(s31, M31, Gamma31)
    return np.abs(A12 + A23 + A31) ** 2


def BW(s, m, Gamma):
    return 1 / (s - m**2 + m * Gamma * 1j)

### Spherical Harmonics (only) Model

```{admonition} Reminder
```{math}
:label: S_H_only
I = 
\left\lvert \sum a_m Y_2^m (\Omega_1) +
\sum b_m Y_1^m (\Omega_2) +
c_0 \right\rvert ^2
```
```

In [ ]:
def SH_model(
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    return (
        np.abs(
            compute_spherical_harmonics12(
                phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
            )
            + compute_spherical_harmonics23(phi2, theta2, b_minus1, b_0, b_plus1)
            + c_0
        )
        ** 2
    )

### Breit-Wigner $\times$ Spherical Harmonics Model

```{admonition} Reminder
```{math}
:label: B_W_S_H_combine
I = \left\lvert A^{12}+A^{23}+A^{31} \right\rvert ^2 =
\left\lvert \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} +
\frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} +
\frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \right\rvert ^2
```
```

In [ ]:
def BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    A12 = BW(s12, M12, Gamma12) * compute_spherical_harmonics12(
        phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
    )
    A23 = BW(s23, M23, Gamma23) * compute_spherical_harmonics23(
        phi2, theta2, b_minus1, b_0, b_plus1
    )
    A31 = BW(s31, M31, Gamma31) * c_0
    return np.abs(A12 + A23 + A31) ** 2

### Full Model

```{admonition} Reminder
```{math}
:label: full_expression
I = 
\left\lvert \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}}\times s^{0.5+0.9u_3} +
\frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}}\times s^{0.5+0.9t_1} +
\frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}}\times s^{1.08+0.2t_2}  \right\rvert ^2
```
```

In [ ]:
def full_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    t1,
    t2,
    u3,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    A12 = (
        BW(s12, M12, Gamma12)
        * compute_spherical_harmonics12(
            phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
        )
        * s12 ** (0.5 + 0.9 * u3)
    )
    A23 = (
        BW(s23, M23, Gamma23)
        * compute_spherical_harmonics23(phi2, theta2, b_minus1, b_0, b_plus1)
        * s23 ** (0.5 + 0.9 * t1)
    )
    A31 = BW(s31, M31, Gamma31) * c_0 * s31 ** (1.08 + 0.2 * t2)
    return np.abs(A12 + A23 + A31) ** 2

### Dalitz Plots of (sub)models

The Dalitz plots of sub-models: Breit-Wigner (only), Spherical Harmonics (only),and Breit-Wigner $\times$
 Spherical Harmonics Model. 
 
 We want to see the different contribution of each term and when combining together.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(R"Dalitz Plots of sub-models")

hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax1.set_xlabel(R"$m^2(\eta \pi^0)$")
ax1.set_title("BW model only")
ax1.set_ylabel(R"$m^2(\pi^0 p)$")

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=SH_model(
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax2.set_title("SH model only")
ax2.set_xlabel(R"$m^2(\eta \pi^0)$")
ax2.set_ylabel(R"$m^2(\pi^0 p)$")


hist3 = ax3.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax3.set_title(R"BW $\times$ SH model")
ax3.set_xlabel(R"$m^2(\eta \pi^0)$")
ax3.set_ylabel(R"$m^2(\pi^0 p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

Set a cut at max=0.15 on the Dalitz plot of BW $\times$ SH model to visualize the "strides" clearer.  

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
fig.suptitle(R"Dalitz Plot of BW $\times$ SH model with cut at max=0.15")
hist = ax.hist2d(
    s12_phsp,
    s23_phsp,
    bins=200,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)

ax.set_xlabel(r"$m^2(\eta \pi^0)$")
ax.set_ylabel(r"$m^2(\pi^0 p)$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1) = plt.subplots(figsize=(3.5, 3), ncols=1, sharey=True)
hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=full_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        t1_phsp,
        t2_phsp,
        u3_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-8,
    density=True,
    cmap="jet",
)
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
ax1.set_title("Full model")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
fig.tight_layout()
fig.show()

In [ ]:
weight_BW = BW_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
)
weight_SH = SH_model(
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    a_0=a_0,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)

weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_0=a_0,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)
weight_Full = full_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    t1_phsp,
    t2_phsp,
    u3_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_0=a_0,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)

## Data Generation

### Hit and miss intensity sample

This function generates a data sample based on a model by using hit and miss filter applied to the phase space sample. 

The output is a `tuple`[$p_1$,$p_2$,$p_3$,$p_a$,$p_b$], which contains five set of 4-vectors.

:::{tip}
The function block below is scrollable.
:::

In [ ]:
def generate_data(
    model: callable, size: int, bunch_size: int | None = None, seed: int | None = None
) -> tuple[MomentumNumpy4D, ...]:
    if bunch_size is None:
        bunch_size = size
    rng = np.random.default_rng(seed=seed)
    generated_seed = rng.integers(1_000_000)
    phase_space = generate_phsp_all(size, generated_seed)
    data_sample = _hit_and_miss_filter(model, phase_space, generated_seed)
    while get_size_data(data_sample) < size:
        phase_space = generate_phsp_all(bunch_size, generated_seed)
        bunch = _hit_and_miss_filter(model, phase_space)
        data_sample = concatenate_data(data_sample, bunch)
    return remove_overflow_data(data_sample, size)


def _hit_and_miss_filter(
    model: callable, phase_space: tuple[MomentumNumpy4D, ...], seed: int | None = None
) -> tuple[MomentumNumpy4D, ...]:
    p1, p2, p3, pa, pb = phase_space
    p12 = p1 + p2
    p23 = p2 + p3
    p31 = p3 + p1

    intensities: np.ndarray = model(
        s12=p12.m2,
        s23=p23.m2,
        s31=p31.m2,
        t1=(pa - p1).m2,
        t2=(pa - p2).m2,
        u3=(pb - p3).m2,
        phi1=phi_helicity(p1, p12),
        theta1=theta_helicity(p1, p12),
        phi2=phi_helicity(p2, p23),
        theta2=theta_helicity(p2, p23),
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    )
    rng = np.random.default_rng(
        seed=seed
    )  # Fixed seed is used here for Reproducibility and Testing
    random_intensities = rng.uniform(0, intensities.max(), size=intensities.shape)
    selector = intensities > random_intensities
    return (
        p1[selector],
        p2[selector],
        p3[selector],
        pa[selector],
        pb[selector],
    )


def get_size_data(
    data: tuple[
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
    ],
):
    return len(data[0])


def concatenate_data(
    data1: tuple[MomentumNumpy4D, ...],
    data2: tuple[MomentumNumpy4D, ...],
) -> tuple[MomentumNumpy4D, ...]:
    return tuple(concatenate_vectors((pi1, pj2)) for pi1, pj2 in zip(data1, data2))


def concatenate_vectors(vectors: tuple[MomentumNumpy4D]) -> MomentumNumpy4D:
    return vector.array({
        "px": np.concatenate([p.px for p in vectors]),
        "py": np.concatenate([p.py for p in vectors]),
        "pz": np.concatenate([p.pz for p in vectors]),
        "E": np.concatenate([p.e for p in vectors]),
    })


def remove_overflow_data(
    data: tuple[MomentumNumpy4D, ...], size: int
) -> tuple[MomentumNumpy4D, ...]:
    return tuple(momentum[:size] for momentum in data)

A few functions are defined above for generating data sample from filtering phase space samples of particle decays and converting them into four-momentum vectors. The main ideas of them are:

- `generate_data()`: The main function used to generate the data 
    - It simulates events until it gathers a specified number of valid data points that meet the model’s conditions.
    - Parameters:
        - model: A callable that computes the intensity of events based on given kinematic variables.
        - size: The desired number of valid events to generate.
        - bunch_size: The number of events to generate per iteration if the initial batch doesn't meet the required size. Defaults to size if not provided.
        - seed: Seed for random number generation to ensure reproducibility.
    - Process:
        - Initialize a random number generator.
        - Generate an initial set of phase space data (phase_space) and filter it using the model to obtain valid events (data_sample).
        - If the filtered data (data_sample) is less than the desired size, additional data is generated in batches (bunch_size) and filtered until the size requirement is met.
        - The function ensures the returned dataset size matches exactly the requested size.
     

- `_hit_and_miss_filter`: The other important function that filters the generated phase space data
    - using the hit-and-miss Monte Carlo method based on the model intensities.
    - Parameters:
        - model: The model function to calculate intensities.
        - phase_space: Tuple containing momentum vectors of particles involved in the event.
        - seed: Seed for random number generation.
    - Process:
        - Calculate invariant masses and other kinematic variables required by the model.
        - Call the model with these variables to compute intensities for each event.
        - Generate random numbers and select events where the model intensity is greater than a random threshold, simulating a probability proportional to the intensity.
     
There are other assisted functions that are used in the functions above:

- `get_size_data`:

  - Simply returns the number of events in the data, used to check if more data needs to be generated.

- `concatenate_data`:
        - Merges two datasets into one, maintaining the structure of tuples of momentum vectors.

- `concatenate_vectors`:
        - Helper function to concatenate individual momentum vectors within the data tuples.

- `remove_overflow_data`:
        - Trims the dataset to the desired size if it exceeds the specified number due to batch generation.

Now we can use the function `generate_data` to generate our data sample:

In [ ]:
%%time
data = generate_data(BW_SH_model, data_events, seed=0)

In [ ]:
p1_data, p2_data, p3_data, pa_data, pb_data = data

In [ ]:
p12_data = p1_data + p2_data
p23_data = p2_data + p3_data
p31_data = p3_data + p1_data

s12_data = p12_data.m2
s23_data = p23_data.m2
s31_data = p31_data.m2
t1_data = (pa_data - p1_data).m2
t2_data = (pa_data - p2_data).m2
u3_data = (pb_data - p3_data).m2

theta1_CM_data = p1_data.theta
theta2_CM_data = p2_data.theta
theta3_CM_data = p3_data.theta
phi1_CM_data = p1_data.phi
phi2_CM_data = p2_data.phi
phi3_CM_data = p3_data.phi

theta1_data = theta_helicity(p1_data, p12_data)
theta2_data = theta_helicity(p2_data, p23_data)
theta3_data = theta_helicity(p3_data, p31_data)
phi1_data = phi_helicity(p1_data, p12_data)
phi2_data = phi_helicity(p2_data, p23_data)
phi3_data = phi_helicity(p3_data, p31_data)

Have a look at the expected Dalitz plot from the generated data sample

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
fig.suptitle("Dalitz Plot of Generated Data (with cut at max=0.15)")
hist = ax.hist2d(
    s12_data, s23_data, bins=100, cmin=1e-6, density=True, cmap="jet", vmax=0.15
)
ax.set_xlabel(r"$m^2_{\eta \pi^0}$")
ax.set_ylabel(r"$m^2_{\pi^0 p}$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
fig.show()

(my_section)=
## 1D projections 

The 1D projection distribution of CM angles, the helicity angles and invariant mass of the model, phase space, and data are shown in this section. 

The angles in phase space, data, and models with good initial guess parameters are all obtained for comparison.

In [ ]:
thetaCM_subtitles = [
    R"$\cos (\theta_{1}^{{CM}}) \equiv \cos (\theta_{\eta}^{{CM}})$",
    R"$\cos (\theta_{2}^{{CM}}) \equiv \cos (\theta_{\pi^0}^{{CM}})$",
    R"$\cos (\theta_{3}^{{CM}}) \equiv \cos (\theta_{p}^{{CM}})$",
]
phiCM_subtitles = [
    R"$\phi_1^{CM} \equiv \phi_{\eta}^{CM}$",
    R"$\phi_2^{CM} \equiv \phi_{\pi^0}^{CM}$",
    R"$\phi_3^{CM} \equiv \phi_{p}^{CM}$",
]
theta_subtitles = [
    R"$\cos (\theta_{1}^{{12}}) \equiv \cos (\theta_{\eta}^{{\eta \pi^0}})$",
    R"$\cos (\theta_{2}^{{23}}) \equiv \cos (\theta_{\pi^0}^{{\pi^0 p}})$",
    R"$\cos (\theta_{3}^{{31}}) \equiv \cos (\theta_{p}^{{p \eta}})$",
]
phi_subtitles = [
    R"$\phi_1^{12} \equiv \phi_{\eta}^{{\eta \pi^0}}$",
    R"$\phi_2^{23} \equiv \phi_{\pi^0}^{{\pi^0 p}}$",
    R"$\phi_3^{31} \equiv \phi_{p}^{{p \eta}}$",
]
mass_subtitles = [
    R"$m_{12} \equiv m_{\eta \pi^0}$",
    R"$m_{23} \equiv m_{\pi^0 p}$",
    R"$m_{31} \equiv m_{p \eta}$",
]

fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax3 in enumerate(theta_ax, 1):
    ax3.set_title(theta_subtitles[i - 1])
    ax3.set_xticks([-1, 0, 1])

for i, ax4 in enumerate(phi_ax, 1):
    ax4.set_title(phi_subtitles[i - 1])
    ax4.set_xticks([-np.pi, 0, np.pi])
    ax4.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax5 in enumerate(mass_ax, 1):
    ax5.set_title(mass_subtitles[i - 1])

thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    bins=100,
    label="data",
    density=True,
)

phi_ax[0].hist(
    phi1_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=100,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()

fig.tight_layout()
plt.show()

## Fitting 

The initial guess of parameters before fittings.
We make a small offset for some default parameters, in an arbitrary choice of values, which is usually the case as an initial guess before fitting.

In this particular situation, the parameters (to be fitted) are mass $M$ ($M_{12}$, $M_{23}$, and $M_{31}$) and decay width $\Gamma$ (($\Gamma_{12}$, $\Gamma_{23}$, and $\Gamma_{31}$)).
While the kinematics variables (Mandelstam variable $s$ ($s_{12}$, $s_{23}$, and $s_{31}$) and helicity angles $\theta$ (($\theta_{12}$, $\theta_{23}$, and $\theta_{31}$)) and $\phi$ (($\phi_{12}$, $\phi_{23}$, and $\phi_{31}$))) are data columns that are computed from the phase space sample.

### Starting values for parameters

Firstly, have a look at the parameter defaults

In [ ]:
toy_parameters

Next, the new starting values of parameter is set by shifting a small value from the parameter defaults 

In [ ]:
guessed_parameters = dict(
    M12=1.4,
    Gamma12=0.15,
    M23=1.59,
    Gamma23=0.05,
    M31=1.82,
    Gamma31=0.2,
    # a_minus2 is set to fixed and not changed during fitting
    a_minus1=0.51,
    a_0=3.49,
    a_plus1=4.08,
    a_plus2=2.6,
    b_minus1=-1.2,
    b_0=4.04,
    b_plus1=0.41,
    c_0=2.66,
)

new_weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    **{
        **toy_parameters,
        **guessed_parameters,
    },
)

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(theta_subtitles[i - 1])
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(phi_subtitles[i - 1])
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(mass_subtitles[i - 1])

data_kwargs = dict(bins=100, label="data", density=True)
data_phi_kwargs = dict(bins=50, label="data", density=True)


thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    **data_kwargs,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    **data_kwargs,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    **data_kwargs,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    **data_kwargs,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    **data_kwargs,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    **data_kwargs,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phi_ax[0].hist(
    phi1_data,
    **data_phi_kwargs,
)
phi_ax[1].hist(
    phi2_data,
    **data_phi_kwargs,
)
phi_ax[2].hist(
    phi3_data,
    **data_phi_kwargs,
)

mass_ax[0].hist(
    p12_data.m,
    **data_kwargs,
)
mass_ax[1].hist(
    p23_data.m,
    **data_kwargs,
)
mass_ax[2].hist(
    p31_data.m,
    **data_kwargs,
)

model_kwargs = dict(
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

model_phi_kwargs = dict(
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    **model_kwargs,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    **model_kwargs,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    **model_kwargs,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    **model_phi_kwargs,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    **model_phi_kwargs,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    **model_phi_kwargs,
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    **model_kwargs,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    **model_kwargs,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    **model_kwargs,
)
phi_ax[0].hist(
    phi1_phsp,
    **model_phi_kwargs,
)
phi_ax[1].hist(
    phi2_phsp,
    **model_phi_kwargs,
)
phi_ax[2].hist(
    phi3_phsp,
    **model_phi_kwargs,
)
mass_ax[0].hist(
    p12_phsp.m,
    **model_kwargs,
)
mass_ax[1].hist(
    p23_phsp.m,
    **model_kwargs,
)
mass_ax[2].hist(
    p31_phsp.m,
    **model_kwargs,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"CM angles, Helicity angles and invariant mass (Before fitting: starting"
    r" parameters for BW $\times$ SH model)",
    y=1,
)
fig.tight_layout()
plt.show()

### Estimator and unbinned Log likelihood function

To perform a fit for parameters estimation by using unbinned log likelihood function, we define a functions:

In [ ]:
def unbinned_nll(
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
) -> float:
    phsp = (
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
    )
    parameter = dict(
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    )
    data = (
        p12_data.m2,
        p23_data.m2,
        p31_data.m2,
        phi1_data,
        theta1_data,
        phi2_data,
        theta2_data,
    )
    model_integral = BW_SH_model(*phsp, **parameter).mean()
    data_intensities = BW_SH_model(*data, **parameter)
    likelihoods = data_intensities / model_integral
    log_likelihood = np.log(likelihoods).sum()
    return -log_likelihood

:::{tip}
The phase space (phsp) is used for the normalization calculation for the unbinned Log likelihood function
:::

The unbinned Log likelihood function with input parameters:

In [ ]:
%%time
unbinned_nll(
    **{
        **toy_parameters,
        **guessed_parameters,
    },
)

### Optimizer

As a final step in this work, it is necessary to go for finding the parameter values that maximize the unbinned Log likelihood function, i.e., performing the optimization. 

This can be performed by e.g. `iminuit` package in python.

In [ ]:
arg_names = tuple(guessed_parameters)
progress_bar = None
global_function_calls = 0


def wrapped_estimator(*args: float) -> float:
    # minuit migrad expects positional-argument functions
    global progress_bar, global_function_calls
    if progress_bar is None:
        progress_bar = tqdm(desc="Running fit")
    progress_bar.update()
    global_function_calls += 1
    new_kwargs = {
        **toy_parameters,
        **dict(zip(arg_names, args)),
    }
    return unbinned_nll(**new_kwargs)

In [ ]:
optimizer = Minuit(
    wrapped_estimator,
    **guessed_parameters,
    name=guessed_parameters,
)
optimizer.errordef = Minuit.LIKELIHOOD
optimizer

In [ ]:
start_time = time.time()
optimizer.migrad()
end_time = time.time()
progress_bar.close()
progress_bar = (
    None  # Ensure to clean up by closing and removing the progress bar object
)

In [ ]:
duration = end_time - start_time
print(
    f"Optimization took {duration:.2f} seconds and {global_function_calls} function"
    " (steps/iterations) calls."
)

### Final fit result

In [ ]:
print("The result after optimization:")
optimizer.params

In [ ]:
optimized_parameters = {p.name: p.value for p in optimizer.params}
rounded_optimized_parameters = {p.name: round(p.value, 3) for p in optimizer.params}

parameters_df = pd.DataFrame({
    "Parameter": list(toy_parameters.keys()),
    "Default Value": list(toy_parameters.values()),
    "Starting Value": [guessed_parameters.get(name) for name in toy_parameters],
    "Optimized Value": [
        rounded_optimized_parameters.get(name) for name in toy_parameters
    ],
})

print("Comparison of Default, Starting, and Optimized (Fitted) Parameters:")
parameters_df

Now we see the fit results of CM angles, Helicity angles and invariant mass as a comparison of data in 1D projection plots

In [ ]:
fitted_weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    **{
        **toy_parameters,
        **optimized_parameters,
    },
)

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(theta_subtitles[i - 1])
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(phi_subtitles[i - 1])
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(mass_subtitles[i - 1])

thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    **data_kwargs,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    **data_kwargs,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    **data_kwargs,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    **data_kwargs,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    **data_kwargs,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    **data_kwargs,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    **data_phi_kwargs,
    alpha=0.5,
)
phi_ax[0].hist(
    phi1_data,
    **data_phi_kwargs,
)
phi_ax[1].hist(
    phi2_data,
    **data_phi_kwargs,
)
phi_ax[2].hist(
    phi3_data,
    **data_phi_kwargs,
)

mass_ax[0].hist(
    p12_data.m,
    **data_kwargs,
)
mass_ax[1].hist(
    p23_data.m,
    **data_kwargs,
)
mass_ax[2].hist(
    p31_data.m,
    **data_kwargs,
)


fitted_kwargs = dict(
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

fitted_phi_kwargs = dict(
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    **fitted_kwargs,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    **fitted_kwargs,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    **fitted_kwargs,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    **fitted_phi_kwargs,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    **fitted_phi_kwargs,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    **fitted_phi_kwargs,
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    **fitted_kwargs,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    **fitted_kwargs,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    **fitted_kwargs,
)
phi_ax[0].hist(
    phi1_phsp,
    **fitted_phi_kwargs,
)
phi_ax[1].hist(
    phi2_phsp,
    **fitted_phi_kwargs,
)
phi_ax[2].hist(
    phi3_phsp,
    **fitted_phi_kwargs,
)
mass_ax[0].hist(
    p12_phsp.m,
    **fitted_kwargs,
)
mass_ax[1].hist(
    p23_phsp.m,
    **fitted_kwargs,
)
mass_ax[2].hist(
    p31_phsp.m,
    **fitted_kwargs,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"CM angles, Helicity angles and invariant mass (After fitting: updated fitted"
    r" parameters for BW $\times$ SH model)",
    y=1,
)
fig.tight_layout()
plt.show()

#### Dalitz plots of phsp, data, and fit result

To conclude our analysis in this document, we examine the Dalitz plots for the following cases: phase space, generated data, default parameters of the model, and fitted parameters of the model.

- Phase Space Dalitz Plot:
    - The plot displays a flat distribution, representing the kinematically allowed region for the reaction. This flat distribution serves as a baseline, showing no dynamical effects or resonant structures.
- Generated Data Dalitz Plot:
    - This plot is derived from synthetic data that simulates the actual experimental conditions. It includes potential resonances (and background processes). The generated data reflects the physical phenomena expected in the real experimental scenario.
- Model with Default Parameters Dalitz Plot:
    - Here, the Dalitz plot illustrates the model's predictions using the initial set of parameters. This provides a theoretical reference for comparison with actual data and the fitted model.
- Fitted Parameters Dalitz Plot:
    - After performing the fitting procedure, this plot shows the model's predictions with optimized parameters. The resonances and their characteristics are clearly visible, indicating the physical information extracted from the data.
 
By comparing these Dalitz plots, we can observe how the model evolves from theoretical predictions to a fitted solution that aligns with the experimental or generated data. The physical information about possible resonances becomes more apparent, allowing us to extract meaningful insights into the underlying processes of the reaction.

This comprehensive analysis showcases the interplay between kinematics and dynamics in understanding particle interactions, highlighting the power of Partial Wave Analysis in uncovering the nuances of hadronic reactions.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle(R"Dalitz Plots of Phase space, generated data")
hist2 = ax2.hist2d(
    s12_data, s23_data, bins=100, cmin=1e-6, density=True, cmap="jet", vmax=0.15
)
ax2.set_xlabel(R"$m^2(\eta \pi)$")
ax2.set_title("Generated data (with cut at max=0.15)")
ax2.set_ylabel(R"$m^2(\pi p)$")

hist1 = ax1.hist2d(s12_phsp, s23_phsp, bins=100, cmin=1e-2, density=True, cmap="jet")
ax1.set_title("Phase space")
ax1.set_xlabel(R"$m^2(\eta \pi)$")
ax1.set_ylabel(R"$m^2(\pi p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)

fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle(R"Dalitz Plots of parameter defaults and fitted parameters")

hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)
ax1.set_title(R"Parameters defaults BW $\times$ SH model (with cut at max=0.15)")
ax1.set_xlabel(R"$m^2(\eta \pi)$")
ax1.set_ylabel(R"$m^2(\pi p)$")

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=fitted_weight_BW_SH,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)
ax2.set_title(R"Fitted parameters BW $\times$ SH model (with cut at max=0.15)")
ax2.set_xlabel(R"$m^2(\eta \pi)$")
ax2.set_ylabel(R"$m^2(\pi p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

# Summary

In this document, we have covered the general workflow of PWA, which consists of three major parts:

As a summary, what have we done in this document consists of three major parts:
- Amplitude Model formulation
    - Introduced the theoretical model for the amplitude of the reaction.
    - Defined the mathematical expressions representing the partial waves and their respective contributions.
    - Incorporated the relevant physical parameters, such as resonance properties. 
- Phase space and data sample generation (or data analysis)
    - Generated phase space distributions for the three-body decay processes.
    - Created synthetic data samples or analyzed experimental data.
    - Ensured that the data accurately reflects the kinematic constraints and dynamics of the reaction. 
- Perform fitting, analyze and evaluate the results
    - Performed a fit of the amplitude model to the generated or experimental data.
    - Used statistical techniques to optimize the model parameters.
    - Analyzed the fit results to extract physical parameters.
    - Evaluated the quality of the fit and the consistency of the model with the data.
    - Interpreted the physical significance of the extracted parameters and identified potential resonances and their properties.

This structured approach provides a comprehensive understanding of both the reaction dynamics and kinematics, and helps in extracting meaningful physical insights from the data.

:::{tip} Extra
have a look for another example of PWA for $J/\psi \to \gamma \pi^0 \pi^0$ by using `ComPWA` at [here](https://tensorwaves.readthedocs.io/stable/amplitude-analysis/) .
:::